# BLEU Implementation

In [199]:
import unittest
from math import exp, log
from functools import reduce
from operator import mul
from nltk.translate.bleu_score import sentence_bleu

In [220]:
cand = "the cat the cat on the mat"
# cand = "the the the the the the the the"
ref_1 = "the cat is on the mat"
ref_2 = "there is a cat on the mat"

In [221]:
cand = cand.split()
ref_1 = ref_1.split()
ref_2 = ref_2.split()

In [222]:
def word_count(str):
    counts = dict()
    words = sorted(str)
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

In [223]:
def mod_ngram_precision_count(cand_sent, ref_list, sum_count=True):
  cand_count = word_count(cand_sent)
  max_ref_count = dict.fromkeys(cand_count, 0)
  ref_counts = [word_count(ref) for ref in ref_list]
  print(cand_count)
  print(ref_counts)

  for ref_count in ref_counts:
    for key, value in ref_count.items():
      if key in max_ref_count:
          if max_ref_count[key] < value: # update max count in refs
            max_ref_count[key] = value
  for key, value in max_ref_count.items():
    if cand_count[key] > value: # if candidate count more than max ref count
      cand_count[key] = value # clip with max ref count
  if sum_count==True:    
    return sum(cand_count.values())
  else:
    return cand_count

In [224]:
mod_ngram_precision_count(cand, [ref_1, ref_2])

{'cat': 2, 'mat': 1, 'on': 1, 'the': 3}
[{'cat': 1, 'is': 1, 'mat': 1, 'on': 1, 'the': 2}, {'a': 1, 'cat': 1, 'is': 1, 'mat': 1, 'on': 1, 'the': 1, 'there': 1}]


5

In [225]:
def ngram(text, gram_count):
  return [tuple(text[i:i+gram_count]) for i in range(len(text)-gram_count+1)]

In [229]:
def bleu(cand_sent, ref_list):
  closest_ref_len = min([len(i) for i in ref_list], key=lambda x:abs(x-len(cand_sent)))
  bp = min(1, exp(1 - closest_ref_len / len(cand_sent)))
  prec = [mod_ngram_precision_count(ngram(cand_sent, i), [ngram(ref, i) for ref in ref_list]) / len(ngram(cand_sent, i)) for i in range(1,5)]
  return bp * reduce(mul, [exp(0.25 * log(prec_n)) if prec_n != 0 else exp(0.25 * 0) for prec_n in prec], 1)

In [227]:
def bleu(cand_sent, ref_list):
  closest_ref_len = min([len(i) for i in ref_list], key=lambda x:abs(x-len(cand_sent)))
  bp = min(1, exp(1 - closest_ref_len / len(cand_sent)))
  prec = [mod_ngram_precision_count(ngram(cand_sent, i), [ngram(ref, i) for ref in ref_list]) / len(ngram(cand_sent, i)) for i in range(1,5)]
  return bp * reduce(mul, [exp(0.25 * log(prec_n)) if prec_n != 0 else 0 for prec_n in prec], 1)

In [230]:
print(bleu(cand, [ref_1, ref_2]))
print(sentence_bleu(references=[ref_1, ref_2], hypothesis=cand, weights=(0.25,0.25,0.25,0.25)))

{('cat',): 2, ('mat',): 1, ('on',): 1, ('the',): 3}
[{('cat',): 1, ('is',): 1, ('mat',): 1, ('on',): 1, ('the',): 2}, {('a',): 1, ('cat',): 1, ('is',): 1, ('mat',): 1, ('on',): 1, ('the',): 1, ('there',): 1}]
{('cat', 'on'): 1, ('cat', 'the'): 1, ('on', 'the'): 1, ('the', 'cat'): 2, ('the', 'mat'): 1}
[{('cat', 'is'): 1, ('is', 'on'): 1, ('on', 'the'): 1, ('the', 'cat'): 1, ('the', 'mat'): 1}, {('a', 'cat'): 1, ('cat', 'on'): 1, ('is', 'a'): 1, ('on', 'the'): 1, ('the', 'mat'): 1, ('there', 'is'): 1}]
{('cat', 'on', 'the'): 1, ('cat', 'the', 'cat'): 1, ('on', 'the', 'mat'): 1, ('the', 'cat', 'on'): 1, ('the', 'cat', 'the'): 1}
[{('cat', 'is', 'on'): 1, ('is', 'on', 'the'): 1, ('on', 'the', 'mat'): 1, ('the', 'cat', 'is'): 1}, {('a', 'cat', 'on'): 1, ('cat', 'on', 'the'): 1, ('is', 'a', 'cat'): 1, ('on', 'the', 'mat'): 1, ('there', 'is', 'a'): 1}]
{('cat', 'on', 'the', 'mat'): 1, ('cat', 'the', 'cat', 'on'): 1, ('the', 'cat', 'on', 'the'): 1, ('the', 'cat', 'the', 'cat'): 1}
[{('cat', '

In [186]:
print(sentence_bleu(references=[ref_1, ref_2], hypothesis=cand, weights=(0.25,0.25,0.25,0.25)))

0.4671379777282001


In [156]:
1 * reduce(mul, [exp(0.25 * log(prec_n)) if prec_n != 0 else 0 for prec_n in [0.25, 0.0, 0.0, 0.0]], 1)

0.0

In [147]:
log(0)

ValueError: math domain error

In [182]:
exp(0.25 * log(0))

ValueError: math domain error

In [183]:
exp(0.25 * 0)

1.0

In [195]:
exp(0)

1.0

In [29]:
well? which I don't quite understand is these two:
bp * reduce(mul, [exp(0.25 * log(prec_n)) if prec_n != 0 else 0 for prec_n in prec], 1)
bp * reduce(mul, [exp(0.25 * log(prec_n)) if prec_n != 0 else exp(0.25 * 0) for prec_n in prec], 1)

from my point of view, if prec_n is 0, each component should be exp(0.25 * 0), which results in 1.
But the first one code simply multiplied with 0 looks correct because the result is same  with the nltk's bleu score.
hmm did I miss somthing? 

In [198]:
0.25 * 0

0.0